# 智能RSI参数

根据个股一年来最低的RSI来判断当前股价是否处于最低位置

In [7]:
import jqdatasdk as jq
import os
import arrow
import datetime
import numpy as np
from jqdatasdk import *
import pandas as pd
import json
import math

account = os.getenv("JQ_ACCOUNT")
password = os.getenv("JQ_PASSWORD")

jq.auth(account, password)
valuations=jq.get_fundamentals(query(valuation))
secs = jq.get_all_securities()
    
def get_name(code):
    return secs[secs.index==code].iloc[0]["display_name"]

def get_valuation(code):
    return int(valuations[valuations.code==code].iloc[0]["market_cap"])

def choose_stocks(exclude_st=True, exclude_688=True, valuation_range=(100,2000)):
    result = secs
    if exclude_688:
        result = result[result.index.str.startswith("688")==False]
    if exclude_st:
        result = result[result.display_name.str.find("ST") == -1]
        
    codes = []
    for code in result.index:
        try:
            if valuation_range[0] <= get_valuation(code) <= valuation_range[1]:
                codes.append(code)
        except Exception:
            pass
            
    return codes

def top_n_argpos(ts: np.array, n: int) -> np.array:
    """get top n (max->min) elements and return argpos which its value ordered in descent

    Example:
        >>> top_n_argpos([4, 3, 9, 8, 5, 2, 1, 0, 6, 7], 2)
        array([2, 3])
    Args:
        ts (np.array): [description]
        n (int): [description]

    Returns:
        np.array: [description]
    """
    return np.argsort(ts)[-n:][::-1]

def moving_average(ts: np.array, win: int):
    """计算时间序列ts在win窗口内的移动平均

    Example:

        >>> ts = np.arange(7)
        >>> moving_average(ts, 5)
        >>> array([2.0000, 3.0000, 4.0000])

    """

    return np.convolve(ts, np.ones(win) / win, "valid")

def relative_strength_index(prices, period=14):
    """
    The Relative Strength Index (RSI) is a momentum oscillator.
    It oscillates between 0 and 100.
    It is considered overbought/oversold when it's over 70/below 30.
    Some traders use 80/20 to be on the safe side.
    RSI becomes more accurate as the calculation period (min_periods)
    increases.
    This can be lowered to increase sensitivity or raised to decrease
    sensitivity.
    10-day RSI is more likely to reach overbought or oversold levels than
    20-day RSI. The look-back parameters also depend on a security's
    volatility.

    Like many momentum oscillators, overbought and oversold readings for RSI
    work best when prices move sideways within a range.

    You can also look for divergence with price.
    If the price has new highs/lows, and the RSI hasn't, expect a reversal.
    Signals can also be generated by looking for failure swings and centerline
    crossovers.

    RSI can also be used to identify the general trend.

    The RSI was developed by J. Welles Wilder and was first introduced in his
    article in the June, 1978 issue of Commodities magazine, now known as
    Futures magazine. It is detailed in his book New Concepts In Technical
    Trading Systems.

    http://www.csidata.com/?page_id=797
    http://stockcharts.com/help/doku.php?id=chart_school:technical_indicators:relative_strength_in

    Input:
      prices ndarray
      period int > 1 and < len(prices) (optional and defaults to 14)

    Output:
      rsis ndarray

    Test:

    >>> import numpy as np
    >>> prices = np.array([44.55, 44.3, 44.36, 43.82, 44.46, 44.96, 45.23,
    ... 45.56, 45.98, 46.22, 46.03, 46.17, 45.75, 46.42, 46.42, 46.14, 46.17,
    ... 46.55, 46.36, 45.78, 46.35, 46.39, 45.85, 46.59, 45.92, 45.49, 44.16,
    ... 44.31, 44.35, 44.7, 43.55, 42.79, 43.26])
    >>> print(rsi(prices))
    [ 70.02141328  65.77440817  66.01226849  68.95536568  65.88342192
      57.46707948  62.532685    62.86690858  55.64975092  62.07502976
      54.39159393  50.10513101  39.68712141  41.17273382  41.5859395
      45.21224077  37.06939108  32.85768734  37.58081218]
    """

    num_prices = len(prices)

    if num_prices < period:
        # show error message
        raise SystemExit("Error: num_prices < period")

    # this could be named gains/losses to save time/memory in the future
    changes = prices[1:] - prices[:-1]
    # num_changes = len(changes)

    rsi_range = num_prices - period

    rsis = np.zeros(rsi_range)

    gains = np.array(changes)
    # assign 0 to all negative values
    masked_gains = gains < 0
    gains[masked_gains] = 0

    losses = np.array(changes)
    # assign 0 to all positive values
    masked_losses = losses > 0
    losses[masked_losses] = 0
    # convert all negatives into positives
    losses *= -1

    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])

    if avg_loss == 0:
        rsis[0] = 100
    else:
        rs = avg_gain / avg_loss
        rsis[0] = 100 - (100 / (1 + rs))

    for idx in range(1, rsi_range):
        avg_gain = (avg_gain * (period - 1) + gains[idx + (period - 1)]) / period
        avg_loss = (avg_loss * (period - 1) + losses[idx + (period - 1)]) / period

        if avg_loss == 0:
            rsis[idx] = 100
        else:
            rs = avg_gain / avg_loss
            rsis[idx] = 100 - (100 / (1 + rs))

    return rsis

def get_bars(code, n, end, unit='1d'):
    fields = ["date", "open", "high", "low", "close", "volume"]
    return jq.get_bars(code, n, unit=unit, end_dt=end, fq_ref_date=end, df=False, include_now=True, fields=fields)

def fillna(ts: np.array):
    """将ts中的NaN替换为其前值

    Args:
        ts (np.array): [description]
    """
    if np.all(np.isnan(ts)):
        raise ValueError("all of ts are NaN")

    if ts[0] is None or math.isnan(ts[0]):
        idx = np.argwhere(~np.isnan(ts))[0]
        ts[0] = ts[idx]

    mask = np.isnan(ts)
    idx = np.where(~mask, np.arange(mask.size), 0)
    np.maximum.accumulate(idx, out=idx)
    return ts[idx]

AssertionError: username is required

In [ ]:
def load_data(path:str):
    df = pd.read_csv(path, sep="\t", usecols=["股票", "代码", "最低", "最低日", "最高", "最高日"])
    df.set_index("代码", inplace=True)
    return df
        
def init_data(save_to:str):
    dirname = os.path.dirname(save_to)
    os.makedirs(dirname, exist_ok=True)
    results = []
    for code in choose_stocks():
        name = get_name(code)
        try:
            bars = get_bars(code, 250, arrow.now().date())
            if len(bars) != 250:
                continue

            frame = bars["date"]
            close = fillna(bars["close"])
            rsi = relative_strength_index(close, 6)
            
            low_pos = np.argmin(rsi) + 6
            low = np.min(rsi)
            low_dt = frame[low_pos]
            
            top_pos = np.argmax(rsi) + 6
            top = np.max(rsi)
            top_dt = frame[top_pos]
            
            results.append((name, code, low, low_dt, top, top_dt))
        except Exception as e:
            print(e)
            pass
        
    df = pd.DataFrame(results, columns=["股票", "代码", "最低", "最低日", "最高", "最高日"])
    df.to_csv(save_to, sep="\t")
            
            
init_data("./rsi.tsv")

In [ ]:
load_data("./rsi.tsv")

,股票,最低,最低日,最高,最高日
代码,,,,,
000009.XSHE,中国宝安,14.782281,2020-12-11,96.546684,2021-06-15
000012.XSHE,南玻A,16.469736,2021-02-05,94.027678,2020-11-16
000016.XSHE,深康佳A,10.829820,2020-09-28,93.185746,2021-08-13
000021.XSHE,深科技,6.984598,2021-05-10,73.978099,2021-06-10
000027.XSHE,深圳能源,16.825453,2021-07-02,89.598780,2020-12-28
...,...,...,...,...,...
603989.XSHG,艾华集团,19.517009,2021-09-02,85.231138,2021-07-22
603993.XSHG,洛阳钼业,10.947313,2021-06-16,92.717150,2021-07-12
603997.XSHG,继峰股份,8.249524,2021-06-30,91.165909,2021-06-01
